Initial_Jrom_Data_Understanding :
1. Dataset diversity - various postures (treading water, breaststroke, backstroke, freestyle, group configurations with 2/3/4 individuals, were included)
2. 8572 labeled images representing drowning, treading water, and swimming
3. 7000 randomly chosen for training set, 1572 allocated to validation set
4. Every image under images folder has a corresponding text file under labels folder with the same name
5. Each text file shows YOLO object detection format: class, x_center, y_center, width, height
6. However, the paper does not explicitly define what each class ID (e.g., 0, 1, 2) represents in the drowning detection task.
7. Based on a labeled image of someone drowning in the paper and observations in the dataset, we can infer that class 2 represents drowning (vertical position with both hands up).
8. Further inference from Figure 5 in the paper suggests that class 0 is swimming and class 1 is tread water

Jishen meeting notes :

Original Training Images -> ROIs -> Training

We need these steps:

1. Human Detection/Segmentation, seperation of ROIs from frames (With YOLO/YOLO-obb)
2. Human Tracking (With SORT)
3. Drowning Detection (With our own model e.g. ViT, CKAN, ResNet..... all CNN-based)
4. Compute Running Threshold (y_t = \alpha y_t + (1 - \alpha) y_(t-1))
5. Draw results

t value > 0.X --> classify as drowning

This value can also be used to compute the colour of the bounding box 

But most of these steps does not involve any classical image processing steps, so maybe we need to include something like

Histrogram Equalisation (To remove the blue background, but there are also pretrains can do this)
Posture Estimation (But not sure how it helps)

Do notice that the ground truth given is in [Prediction_Outcome, Bounding Box Coordinates]

# Train YOLO Model
Warning : Running the training code using YOLO train function will generate a lot of files in the directory

In [1]:
import os
import torch
import cv2
from tqdm.notebook import tqdm
from ultralytics import YOLO

In [2]:
# Check if CUDA gpu is available
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    print("Using CPU")

CUDA available: True
GPU: NVIDIA GeForce RTX 4060 Laptop GPU


In [6]:
# Project file paths
notebook_dir = os.getcwd()  # Current directory (notebooks)
project_root = os.path.dirname(notebook_dir)  # go up one level to get project root
data_dir = os.path.join(project_root, "data")
image_dir = os.path.join(data_dir, "images")
label_dir = os.path.join(data_dir, "labels")
model_dir = os.path.join(project_root, "models")

print(f"Project root: {project_root}")
print(f"Data directory: {data_dir}")
print(f"Model directory: {model_dir}")

Project root: C:\Users\jrom\DataspellProjects\Drowning-Detection
Data directory: C:\Users\jrom\DataspellProjects\Drowning-Detection\data
Model directory: C:\Users\jrom\DataspellProjects\Drowning-Detection\models


In [7]:
class_names = {0: "swimming", 1: "tread water", 2: "drowning"}

## 1. Prepare data for YOLO training
create yaml file for the dataset

In [8]:
def create_data_yaml():
    yaml_content = f"""
            # Dataset paths
            path: {data_dir}  # dataset root directory
            train: {os.path.join("images", "train")}  # train images relative to path
            val: {os.path.join("images", "val")}  # val images relative to path

            # Classes
            names:
              0: swimming
              1: tread water
              2: drowning
            """
    yaml_path = os.path.join(data_dir, "drowning_data.yaml")
    with open(yaml_path, "w") as f:
        f.write(yaml_content)
    print(f"Created data configuration at {yaml_path}")
    return yaml_path

# Create the data.yaml file
data_yaml = create_data_yaml()

Created data configuration at C:\Users\jrom\DataspellProjects\Drowning-Detection\data\drowning_data.yaml


## 2. Train YOLO11n model

In [15]:
def train_yolo_model(data_yaml, epochs=50, imgsz=640, batch_size=16):
    print("Loading YOLO11n model...")
    model = YOLO("yolo11n.pt")

    print(f"Training model for {epochs} epochs...")
    results = model.train(
        data=data_yaml,
        epochs=epochs,
        imgsz=imgsz,
        batch=batch_size,
        device=0 if torch.cuda.is_available() else 'cpu',
        project=model_dir,
        name="drowning_detection",
        exist_ok=True
    )

    # Save the trained model
    model_path = os.path.join(model_dir, "drowning_detection_yolo11n.pt")
    model.save(model_path)
    print(f"Model saved to {model_path}")

    return model, results

## 3. Validate the model

In [3]:
def validate_model(data_yaml, batch_size=4):  # Remove model argument
    print("Loading best model for validation...")
    model = YOLO(os.path.join(model_dir, "drowning_detection", "weights", "best.pt"))  # Load best model
    
    print("Validating model...")
    results = model.val(data=data_yaml, batch=batch_size, device=0 if torch.cuda.is_available() else 'cpu')

    # Print metrics
    metrics = results.box
    print("\nValidation Results:")
    print(f"mAP50: {metrics.map50.mean():.4f}")
    print(f"mAP50-95: {metrics.map.mean():.4f}")
    print(f"Precision: {metrics.p.mean():.4f}")
    print(f"Recall: {metrics.r.mean():.4f}")

    return results


## 4. Extract ROIs & save to new folder
Only if satisfied with the performance

In [17]:
def extract_and_save_rois(model, confidence=0.5):
    # Create output directory for ROIs
    roi_dir = os.path.join(data_dir, "roi_extracts")
    os.makedirs(roi_dir, exist_ok=True)

    # Create subdirectories for each class
    for class_id, class_name in class_names.items():
        os.makedirs(os.path.join(roi_dir, class_name), exist_ok=True)

    # Path to validation images
    val_img_dir = os.path.join(image_dir, "val")

    # Process each image in the validation set
    image_files = [f for f in os.listdir(val_img_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]

    print(f"Extracting ROIs from {len(image_files)} images...")
    for img_file in tqdm(image_files):
        img_path = os.path.join(val_img_dir, img_file)

        # Run inference on the image
        results = model(img_path)

        # Process the results
        for result in results:
            img = cv2.imread(img_path)
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Extract each detection
            boxes = result.boxes
            for i, box in enumerate(boxes):
                # Get detection details
                cls_id = int(box.cls[0].item())
                conf = box.conf[0].item()

                # Skip detections with low confidence
                if conf < confidence:
                    continue

                # Get bounding box coordinates
                x1, y1, x2, y2 = map(int, box.xyxy[0])

                # Extract ROI
                roi = img_rgb[y1:y2, x1:x2]

                # Skip if ROI is empty
                if roi.size == 0:
                    continue

                # Create filename for ROI
                base_name = os.path.splitext(img_file)[0]
                roi_filename = f"{base_name}_roi{i}_class{cls_id}_{class_names[cls_id]}.jpg"

                # Save ROI to appropriate class folder
                roi_path = os.path.join(roi_dir, class_names[cls_id], roi_filename)
                cv2.imwrite(roi_path, cv2.cvtColor(roi, cv2.COLOR_RGB2BGR))

    print(f"ROIs extracted and saved to {roi_dir}")
    return roi_dir

# Run all steps

In [18]:
# 2. Train the model
model, training_results = train_yolo_model(data_yaml, epochs=50)

Loading YOLO11n model...


100%|██████████| 5.35M/5.35M [00:00<00:00, 13.1MB/s]


Training model for 50 epochs...
New https://pypi.org/project/ultralytics/8.3.92 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.79  Python-3.12.3 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=C:\Users\jrom\DataspellProjects\Drowning-Detection\data\drowning_data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=C:\Users\jrom\DataspellProjects\Drowning-Detection\models, name=drowning_detection, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source

train: Scanning C:\Users\jrom\DataspellProjects\Drowning-Detection\data\labels\train... 7000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7000/7000 [00:04<00:00, 1430.86it/s]


train: New cache created: C:\Users\jrom\DataspellProjects\Drowning-Detection\data\labels\train.cache


...

      50/50      4.21G      0.943     0.4338     0.9529         11        640: 100%|██████████| 438/438 [01:22<00:00,  5.29it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 50/50 [00:12<00:00,  4.15it/s]

                   all       1572       2317      0.974      0.974      0.987      0.748



50 epochs completed in 1.380 hours.
Optimizer stripped from C:\Users\jrom\DataspellProjects\Drowning-Detection\models\drowning_detection\weights\last.pt, 5.4MB
Optimizer stripped from C:\Users\jrom\DataspellProjects\Drowning-Detection\models\drowning_detection\weights\best.pt, 5.4MB

Validating C:\Users\jrom\DataspellProjects\Drowning-Detection\models\drowning_detection\weights\best.pt...
Ultralytics 8.3.79  Python-3.12.3 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
YOLO11n summary (fused): 100 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 50/50 [00:13<00:00,  3.83it/s]


                   all       1572       2317      0.974      0.974      0.987      0.749
              swimming        991       1381      0.987      0.993      0.994      0.832
           tread water        502        571      0.965      0.974      0.986      0.761
              drowning        353        365      0.968      0.956      0.979      0.653
Speed: 0.2ms preprocess, 1.7ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to C:\Users\jrom\DataspellProjects\Drowning-Detection\models\drowning_detection
Model saved to C:\Users\jrom\DataspellProjects\Drowning-Detection\models\drowning_detection_yolo11n.pt


In [9]:
# 3. Validate the model
validation_results = validate_model(data_yaml)

Loading best model for validation...
Validating model...
Ultralytics 8.3.79  Python-3.12.3 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
YOLO11n summary (fused): 100 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning C:\Users\jrom\DataspellProjects\Drowning-Detection\data\labels\val.cache... 1572 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1572/1572 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 393/393 [00:16<00:00, 23.56it/s]


                   all       1572       2317      0.974      0.974      0.987      0.749
              swimming        991       1381      0.987      0.993      0.994      0.831
           tread water        502        571      0.965      0.974      0.986      0.762
              drowning        353        365      0.969      0.956      0.979      0.653
Speed: 0.3ms preprocess, 4.3ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs\detect\val

Validation Results:
mAP50: 0.9867
mAP50-95: 0.7486
Precision: 0.9738
Recall: 0.9745


high accuracy (mAP50: 98.67%), moderate generalisation across IoU thresholds (mAP50-95: 74.86%), strong precision (97.38%), and high recall (97.45%). It detects drowning instances effectively with minimal false positives and false negatives. 

In [ ]:
# 4. Extract ROIs
roi_dir = extract_and_save_rois(model, confidence=0.5)